# **Open Circuit Potential (OCP) Balancing**

This notebook is created to provide an example of using PyBaMM-param package when balancing OCP curve with shifting and stretching the fitted curve to experimental data. This can also be found in `scripts/ocp_balance.py` as Python function.

In [ ]:
%pip install git+https://github.com/paramm-team/pybamm-param.git -q
import pbparam
import pandas as pd
import os

os.chdir(os.path.join(pbparam.__path__[0], "input", "data"))

We have 4 different files in this example. 2 for half-cell and 2 for three-electrode full cell. They are divided for their lithiation and delithiation stage.

In [ ]:
#  .csv files are uploaded
anode_half_lit = pd.read_csv(os.path.join("anode_OCP_2_lit.csv"))
anode_half_delit = pd.read_csv(os.path.join("anode_OCP_2_delit.csv"))
anode_three_lit = pd.read_csv(os.path.join("anode_OCP_3_lit.csv"))
anode_three_delit = pd.read_csv(os.path.join("anode_OCP_3_delit.csv"))

Then, optimisation problem should be passed to pbparam with the data to be used as below. Cost function (default=pbparam.RMSE()) and weights (default=[1,1]) are optional arguments. Weights can be defined as single value list or same lengt with data.

In [ ]:
ocp_balance = pbparam.OCPBalance(
    [anode_three_lit, anode_three_delit],
    [anode_half_lit, anode_half_delit],
    cost_function=pbparam.MLE(),
    weights=[2],
)

Optimiser to minimise cost function should be selected in the nest step as below.

In [ ]:
optimiser = pbparam.ScipyMinimize(method="Nelder-Mead")

```optimiser.optimise(optimisation_problem)``` should be used to run minimasiation algorithm.

In [ ]:
result = optimiser.optimise(ocp_balance)

shift and stretch values and final cost function value can be printed as below.

In [ ]:
print(result)

PyBaMM-param has built in plot function to represent results.

In [ ]:
result.plot();